In [1]:
import json
import datetime as dt
from pathlib import Path

import pandas as pd

In [2]:
DATA_DIR = Path("./data")
input_data = DATA_DIR / "openweather-history-seattle.json"

In [3]:
data = json.loads(input_data.read_text())
len(data)

371376

In [4]:
def format_rows(rows: [dict]) -> [dict]:
    for row in rows:
        weather = row["weather"] if () else []
        d = {k: v for k, v in row.items() if not isinstance(v,(dict,list))}
        d = {**d,**{f"main_{k}": v for k, v in row.get("main",{}).items()}}
        d = {**d,**{f"wind_{k}": v for k, v in row.get("wind",{}).items()}}
        d = {**d,**{f"clouds_{k}": v for k, v in row.get("clouds",{}).items()}}
        d = {**d,**{f"weather_{k}": v for k, v in row["weather"][0].items()}}
        yield d

In [5]:
df = pd.DataFrame(list(format_rows(data)))
df.head()

,dt,dt_iso,timezone,lat,city_name,lon,main_temp,main_temp_min,main_temp_max,main_feels_like,main_pressure,main_humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,283996800,1979-01-01 00:00:00 +0000 UTC,-28800,47.606209,Seattle,-122.332071,17.24,12.94,21.06,7.61,1039,46,5.91,56,99,804,Clouds,overcast clouds,04d
1,284000400,1979-01-01 01:00:00 +0000 UTC,-28800,47.606209,Seattle,-122.332071,16.90,11.84,21.33,6.76,1039,46,6.78,63,100,804,Clouds,overcast clouds,04n
2,284004000,1979-01-01 02:00:00 +0000 UTC,-28800,47.606209,Seattle,-122.332071,16.56,11.62,20.93,6.78,1039,46,6.11,66,53,803,Clouds,broken clouds,04n
3,284007600,1979-01-01 03:00:00 +0000 UTC,-28800,47.606209,Seattle,-122.332071,8.62,3.64,13.08,-1.28,1040,52,6.04,72,0,800,Clear,sky is clear,01n
4,284011200,1979-01-01 04:00:00 +0000 UTC,-28800,47.606209,Seattle,-122.332071,8.15,2.75,12.93,-1.73,1040,53,5.99,80,0,800,Clear,sky is clear,01n


In [6]:
df["dt_utc"] = df.dt_iso.apply(lambda d: dt.datetime.strptime(d,"%Y-%m-%d %H:%M:%S %z %Z"))
df["dt_local"] = df.dt_utc + df.timezone.apply(lambda s: dt.timedelta(seconds=s))
df = df.drop(columns=["dt_iso","timezone","dt"])
df.head()

,lat,city_name,lon,main_temp,main_temp_min,main_temp_max,main_feels_like,main_pressure,main_humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,weather_description,weather_icon,dt_utc,dt_local
0,47.606209,Seattle,-122.332071,17.24,12.94,21.06,7.61,1039,46,5.91,56,99,804,Clouds,overcast clouds,04d,1979-01-01 00:00:00+00:00,1978-12-31 16:00:00+00:00
1,47.606209,Seattle,-122.332071,16.90,11.84,21.33,6.76,1039,46,6.78,63,100,804,Clouds,overcast clouds,04n,1979-01-01 01:00:00+00:00,1978-12-31 17:00:00+00:00
2,47.606209,Seattle,-122.332071,16.56,11.62,20.93,6.78,1039,46,6.11,66,53,803,Clouds,broken clouds,04n,1979-01-01 02:00:00+00:00,1978-12-31 18:00:00+00:00
3,47.606209,Seattle,-122.332071,8.62,3.64,13.08,-1.28,1040,52,6.04,72,0,800,Clear,sky is clear,01n,1979-01-01 03:00:00+00:00,1978-12-31 19:00:00+00:00
4,47.606209,Seattle,-122.332071,8.15,2.75,12.93,-1.73,1040,53,5.99,80,0,800,Clear,sky is clear,01n,1979-01-01 04:00:00+00:00,1978-12-31 20:00:00+00:00


In [7]:
df.to_csv(DATA_DIR/"tidy-data.csv", index=False)